### 目标
    问题：任意手写数字，识别出对应的数字是多少
    要求：使用pytorch构建神经网络,实现KMNIST数据集的训练
    分析：
        1. KMNIST数据集 
                训练集60000张，测试集10000张 
                每张图片大小28*28 
                分10类-对应数字 0 到 9 这 10 个手写数字类别
        2. 构建神经网络
                输入层：(60000, 784)
                隐藏层：(784, 32)
                输出层：(, 10)
        思路：① 60000 张图片组成的训练集对模型进行训练，不断迭代优化模型的参数，获得一组能使模型在整个训练集上
                表现最优的全局参数；
              ② 训练过程中，每张输入的图片数据会依次经过隐藏层的所有 32 个神经元进行特征提取和转换；
              ③ 隐藏层的 32 个神经元的输出结果会作为输入传递到输出层，最终由输出层输出 10 个类别的预测概率，
                对应数字 0 到 9 这 10 个手写数字类别。

    步骤：
        1. 数据预处理
        2. 构建模型
        3. 构建损失函数
        4. 构建优化器
        5. 训练模型
        6. 预测数据

####  1. 数据预处理

In [ ]:
# 导包
import torch
import torch.nn as nn
from torchvision.datasets import KMNIST
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

In [41]:
# 参数初始化
BATCH_SIZE = 50
lr = 0.01
epochs = 20

In [42]:
# 加载训练数据集
train_data = KMNIST(root='./kdata', train=True, download=True, transform=ToTensor())
# 拆分训练数据集
train_dl = DataLoader(train_data, batch_size=BATCH_SIZE)

# 加载测试数据集
test_data = KMNIST(root='./kdata', train=False, download=True, transform=ToTensor())
# 拆分测试数据集
test_dl = DataLoader(test_data, batch_size=BATCH_SIZE)

### 2.构建模型

In [43]:
model = nn.Sequential(
    nn.Linear(784, 32),
    nn.Sigmoid(),
    nn.Linear(32, 10)
)

### 3. 构建损失函数

In [44]:
loss_fn = nn.CrossEntropyLoss()

### 4.构建优化器

In [45]:
optimizer = torch.optim.SGD(model.parameters(), lr)

### 5.训练模型

In [46]:
for epoch in range(epochs):
    for x, y in train_dl:
        y_hat = model(x.reshape(-1, 784))
        loss = loss_fn(y_hat, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"epoch: {epoch}  loss: {loss.item()}")

epoch: 0  loss: 1.9823265075683594
epoch: 1  loss: 1.6238467693328857
epoch: 2  loss: 1.3155268430709839
epoch: 3  loss: 1.0984141826629639
epoch: 4  loss: 0.9566939473152161
epoch: 5  loss: 0.8597520589828491
epoch: 6  loss: 0.7894555926322937
epoch: 7  loss: 0.7363988757133484
epoch: 8  loss: 0.6954023241996765
epoch: 9  loss: 0.6631824970245361
epoch: 10  loss: 0.6373887062072754
epoch: 11  loss: 0.6162920594215393
epoch: 12  loss: 0.5986469388008118
epoch: 13  loss: 0.5835732817649841
epoch: 14  loss: 0.570449948310852
epoch: 15  loss: 0.5588335990905762
epoch: 16  loss: 0.548402726650238
epoch: 17  loss: 0.5389223694801331
epoch: 18  loss: 0.5302198529243469
epoch: 19  loss: 0.5221675038337708


### 6. 预测数据

In [50]:
# 总样本数
total = 0
# 正确分类数
correct = 0

with torch.no_grad():
    for x, y in test_dl:
        y_hat = model(x.reshape(-1, 784)) # 每个图片10个类别的概率值组成
        max_val, max_idx = torch.max(y_hat, 1) # 找到概率值最大的类别索引
        total += y.size(0)
        correct += (max_idx == y).sum().item() # 统计正确分类的个数
print(f'Accuracy: {correct / total}')

Accuracy: 0.7062
